In [49]:
import os

In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [51]:
from Chicken_disease_classification.constants import *
from Chicken_disease_classification.utils.common import read_yaml , create_directories

In [52]:
from pathlib import Path

config_file_path = Path("config/config.yaml")
params_file_path = Path("params.yaml")

In [53]:
class ConfigurationManager:
    def __init__(self , config_filepath = config_file_path , params_filepath = params_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self):
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    

In [54]:
import os 
import urllib.request as request
import zipfile
from Chicken_disease_classification import logger
from Chicken_disease_classification.utils.common import get_size 


In [55]:
class DataIngestion:
    def __init__(self , config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url=self.config.source_URL , 
                filename= self.config.local_data_file
            )
            
            logger.info(f"{filename} downlaod with followong info : \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip:
            zip.extractall(unzip_path)



In [56]:
try:
    
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e :
    raise e 

[2023-10-31 00:49:23,564 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-10-31 00:49:23,566 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-10-31 00:49:23,567 : INFO : common : created directory at: artifacts]
[2023-10-31 00:49:23,568 : INFO : common : created directory at: artifacts/data_ingestion]


[2023-10-31 00:49:27,663 : INFO : 3925016016 : artifacts/data_ingestion/data.zip downlaod with followong info : 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FA40:1EA811:13CA9:21156:654001BA
Accept-Ranges: bytes
Date: Mon, 30 Oct 2023 19:19:27 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4728-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1698693566.489557,VS0,VE879
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1d0cdd27c426650253e315acb83704b24e5c9ed9
Expires: Mon, 30 Oct 2023 19:24:27 GMT
Source-Age: 1

]
